# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet'])

import re
import numpy as np
import pandas as pd
import pickle

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sqlalchemy import create_engine

import warnings
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterMsgDB.db')
df = pd.read_sql_table('DisasterMsgTable', engine)  
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
print(df.shape)

(26028, 39)


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """Method to normalize-->tokenize-->lemmatize-->Stem
    
    Args:
    text: String. Message data
       
    Returns:
    cleaned_data: A list of cleaned strings
    """
    
    # Convert text to lowercase and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize the text data
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok)
        clean_tok = re.sub(r"[^a-zA-Z0-9]", " ", clean_tok.lower())
        clean_tok = clean_tok.strip()
        
        clean_tokens.append(clean_tok)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=42)

# Verify post train/test split that each column has 2 classes
for column in Y_train:
    print(np.unique(Y_train[column]), column)
print()
for column in Y_test:
    print(np.unique(Y_test[column]), column)

[0 1] related
[0 1] request
[0 1] offer
[0 1] aid_related
[0 1] medical_help
[0 1] medical_products
[0 1] search_and_rescue
[0 1] security
[0 1] military
[0 1] water
[0 1] food
[0 1] shelter
[0 1] clothing
[0 1] money
[0 1] missing_people
[0 1] refugees
[0 1] death
[0 1] other_aid
[0 1] infrastructure_related
[0 1] transport
[0 1] buildings
[0 1] electricity
[0 1] tools
[0 1] hospitals
[0 1] shops
[0 1] aid_centers
[0 1] other_infrastructure
[0 1] weather_related
[0 1] floods
[0 1] storm
[0 1] fire
[0 1] earthquake
[0 1] cold
[0 1] other_weather
[0 1] direct_report

[0 1] related
[0 1] request
[0 1] offer
[0 1] aid_related
[0 1] medical_help
[0 1] medical_products
[0 1] search_and_rescue
[0 1] security
[0 1] military
[0 1] water
[0 1] food
[0 1] shelter
[0 1] clothing
[0 1] money
[0 1] missing_people
[0 1] refugees
[0 1] death
[0 1] other_aid
[0 1] infrastructure_related
[0 1] transport
[0 1] buildings
[0 1] electricity
[0 1] tools
[0 1] hospitals
[0 1] shops
[0 1] aid_centers
[0 1] ot

In [6]:
np.random.seed(101)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def display_metrics(actual, predicted, columns):
    """Calculate and disply precision, recall, f1-score, accuracy
    
    Args:
    actual: array. actual label values.
    predicted: array. predicted label values.
    columns: list. List of field names
       
    Returns: None
    """
    
    classify = classification_report(actual, predicted, target_names=columns)    
    print('Classification Report\n', '-' * 75)
    print(classify, '\n')
    
    # Display accuracy
    accuracy = (actual == predicted).mean()
    print('Accuracy\n', '-' * 75)
    print(accuracy)
    print('\nAverage Accuracy:', accuracy.mean())
    

In [8]:
# Test the model on the training data
y_pred = pipeline.predict(X_train)
display_metrics(Y_train, y_pred, y.columns)

Classification Report
 ---------------------------------------------------------------------------
                        precision    recall  f1-score   support

               related       0.99      1.00      0.99     14968
               request       1.00      0.93      0.96      3370
                 offer       1.00      0.73      0.84        92
           aid_related       1.00      0.97      0.98      8146
          medical_help       1.00      0.87      0.93      1542
      medical_products       1.00      0.85      0.92       966
     search_and_rescue       1.00      0.82      0.90       532
              security       1.00      0.74      0.85       338
              military       1.00      0.85      0.92       635
                 water       1.00      0.91      0.95      1242
                  food       1.00      0.92      0.96      2192
               shelter       1.00      0.92      0.96      1716
              clothing       1.00      0.87      0.93       304
    

In [9]:
# Test the model on the test data
y_pred = pipeline.predict(X_test)
display_metrics(Y_test, y_pred, y.columns)

Classification Report
 ---------------------------------------------------------------------------
                        precision    recall  f1-score   support

               related       0.82      0.93      0.87      4938
               request       0.82      0.40      0.54      1104
                 offer       0.00      0.00      0.00        26
           aid_related       0.75      0.53      0.62      2714
          medical_help       0.61      0.06      0.12       542
      medical_products       0.84      0.06      0.11       347
     search_and_rescue       0.67      0.02      0.04       192
              security       0.29      0.02      0.03       133
              military       0.53      0.04      0.08       225
                 water       0.79      0.14      0.24       430
                  food       0.84      0.23      0.36       731
               shelter       0.79      0.23      0.35       598
              clothing       0.90      0.09      0.16       101
    

### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
# Create the grid search parameters
parameters = {'vect__max_df': (0.5, 1.0),
              #'vect__ngram_range': ((1, 1), (1, 2)),  #unigrams or bigrams
              'tfidf__use_idf':[True, False],
              #'tfidf__norm': ('l1', 'l2'),
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5]}

gs = GridSearchCV(pipeline, param_grid=parameters)

np.random.seed(101)
model = gs.fit(X_train, Y_train)
model.best_params_

{'clf__estimator__min_samples_split': 5,
 'clf__estimator__n_estimators': 25,
 'tfidf__use_idf': False,
 'vect__max_df': 0.5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
# Test the model on the test data
y_pred = model.predict(X_test)
display_metrics(Y_test, y_pred, y.columns)

Classification Report
 ---------------------------------------------------------------------------
                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      4938
               request       0.86      0.44      0.58      1104
                 offer       0.00      0.00      0.00        26
           aid_related       0.78      0.64      0.70      2714
          medical_help       0.71      0.06      0.12       542
      medical_products       0.72      0.07      0.14       347
     search_and_rescue       0.83      0.03      0.05       192
              security       0.50      0.01      0.01       133
              military       0.71      0.11      0.19       225
                 water       0.86      0.25      0.39       430
                  food       0.84      0.47      0.60       731
               shelter       0.81      0.20      0.33       598
              clothing       0.62      0.05      0.09       101
    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
# Use a SVC algorithm with default settings
svc_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC()))
])

np.random.seed(101)
svc_pipeline.fit(X_train, Y_train)
y_pred = svc_pipeline.predict(X_test)

display_metrics(Y_test, y_pred, y.columns)

Classification Report
 ---------------------------------------------------------------------------
                        precision    recall  f1-score   support

               related       0.76      1.00      0.86      4938
               request       0.00      0.00      0.00      1104
                 offer       0.00      0.00      0.00        26
           aid_related       0.00      0.00      0.00      2714
          medical_help       0.00      0.00      0.00       542
      medical_products       0.00      0.00      0.00       347
     search_and_rescue       0.00      0.00      0.00       192
              security       0.00      0.00      0.00       133
              military       0.00      0.00      0.00       225
                 water       0.00      0.00      0.00       430
                  food       0.00      0.00      0.00       731
               shelter       0.00      0.00      0.00       598
              clothing       0.00      0.00      0.00       101
    

Just used the default settings as it takes an extremely long time to train the algorithm on the Udacity VMs. What we saw was ok results; however, they were not as good as the RandomForestClassifier.  We'll explore the RandomForestClassifier further by adding an extra feature for the length of the message.

In [19]:
class MessageLength(BaseEstimator, TransformerMixin):

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(len)
        return pd.DataFrame(X_tagged)

In [20]:
pipeline2 = Pipeline([
('features', FeatureUnion([

    ('nlp_pipeline', Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer())
    ])),

    ('txt_len', MessageLength())
])),

('clf', RandomForestClassifier())
])

np.random.seed(101)
pipeline2.fit(X_train, Y_train)
y_pred = pipeline2.predict(X_test)
display_metrics(Y_test, y_pred, y.columns)

Classification Report
 ---------------------------------------------------------------------------
                        precision    recall  f1-score   support

               related       0.85      0.90      0.87      4938
               request       0.82      0.40      0.54      1104
                 offer       0.00      0.00      0.00        26
           aid_related       0.80      0.43      0.56      2714
          medical_help       0.62      0.01      0.02       542
      medical_products       0.67      0.02      0.04       347
     search_and_rescue       0.67      0.01      0.02       192
              security       0.50      0.01      0.01       133
              military       0.50      0.01      0.02       225
                 water       0.90      0.13      0.23       430
                  food       0.85      0.30      0.44       731
               shelter       0.88      0.12      0.22       598
              clothing       0.50      0.04      0.07       101
    

### 9. Export your model as a pickle file

In [21]:
pickle.dump(model, open('ml_model.pickle', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.